# Starting with AWS: IAM Roles and Policies

Cloud access should always follow the principle of **least privilege**. Rather than using root or master credentials for daily development or automation tasks, AWS recommends setting up dedicated IAM users, assigning only the permissions they need, and accessing resources via securely managed credentials or role assumptions.

This notebook walks through the secure setup of AWS IAM for programmatic access using users, groups, policies, and roles. This is the first and most critical step for any team or developer starting with AWS via APIs, SDKs, or the CLI.

We will also explain the structure of IAM, the AWS authentication model, and prepare a minimal user setup that allows us to continue with the rest of the tutorial series securely.

In this notebook, we will:
- Creating a group with limited permissions (VPC and EC2 admin)
- Creating a user assigned to that group
- Enabling programmatic access (access key & secret)
- Attaching a custom policy or AWS-managed policy
- Creating a named profile using the AWS SDKs
- Setting up MFA

## AWS IAM: Identities, Policies, and Roles

Before we begin setup, we need to understand a few key concepts:

- **IAM Users**:  
Identities for people or services needing programmatic access (via access keys) or console access (via password).
- **IAM Groups**:  
Collections of users with shared permissions.
- **IAM Policies**:  
JSON documents defining permissions (read, write, list, deny, etc.).
- **IAM Roles**:  
Delegated identities assumed by users, services, or external accounts, often used for temporary credentials and cross-account access.

In production-grade setups, you'd avoid long-lived IAM users where possible, and instead rely on:
- **Federated login**  
  Allows users from external identity providers (like Azure AD or Google Workspace) to access AWS using their existing credentials, without needing IAM users.

- **Assumed roles**  
  Lets users or services temporarily switch to a role with specific permissions, often used for cross-account access or automation.

- **Temporary credentials**  
  Short-lived credentials automatically managed by AWS, issued to federated users or services like EC2 and Lambda via attached roles.


## Pre-requisites

To follow these instructions, you must log in to the AWS console using a **root user only once** to perform administrative IAM setup.
After that, root credentials should be safely stored and never used for daily operations.


## Using the AWS Console to Set Up IAM Securely

The **AWS Management Console** is the web-based interface for managing AWS services. It provides a graphical UI to configure, deploy, and monitor your resources without writing code or using the CLI. All IAM-related actions, users, roles, policies, can be handled within the **IAM** section of the console.

<img src="assets/pics/000_iam_user_roles_and_policies/aws-console.png" alt="Alt text" width="800rem"/>

Find the AWS Console [here](https://aws.amazon.com/console/).

In the console, you can use the search functionality to find different resources inside your account.

IAM (Identity and Access Management) is the category used to manage authentication and authorization in AWS. You can find it by searching for "IAM" in the AWS Console search bar or selecting it from the "Security, Identity, & Compliance" category.


## Understanding IAM Groups, Users, Roles and Policies

Before we begin creating users and assigning permissions in AWS, it’s essential to understand the core identity and access management (IAM) building blocks. These concepts are foundational not just to AWS, but to secure system design in general.

<img src="assets/pics/000_iam_user_roles_and_policies/users-groups-roles-policies.png" alt="Alt text" width="600rem"/>

- **IAM Users**    
    An IAM user represents a single identity with a unique set of credentials and permissions. This identity can be used to access AWS either via the console (with a username/password) or programmatically (with an access key and secret).

    > **Analogy**: Think of a user account on a Linux system. Just as each user has a unique ID and credentials to log in and execute commands, an IAM user is a uniquely identifiable entity in your AWS environment.

    IAM users should be created only when absolutely necessary — typically for developers or automation scripts — and always with the least privilege necessary to do their job.

- **IAM Groups**    
    An IAM group is a collection of users who share the same set of permissions. Policies are attached to groups, not to individual users. Any user added to the group inherits those permissions automatically.

    > **Analogy**: In Microsoft Active Directory (or any corporate directory), users are placed into groups like "HR", "Finance", or "IT", and access is granted at the group level. IAM groups follow the same model — they simplify permission management by applying access rules collectively.

    Groups have no credentials of their own; they simply provide a way to manage permissions efficiently and consistently across users.

- **IAM Roles**  
    An IAM role is a temporary identity with specific permissions. Unlike a user, a role has no long-term credentials and is meant to be assumed by users, AWS services, or external identities. AWS automatically issues and rotates the temporary credentials when a role is assumed.

    > **Analogy**: Think of a visitor badge in a secure office. An employee swipes their badge to get in every day, but a guest is given a temporary visitor pass that allows limited access for a short period. IAM roles serve this same purpose: temporary, scoped access without permanent credentials.

    Roles are essential for automation (e.g., EC2 instances writing to S3), delegation, and secure cross-account or external access.

- **IAM Policies**    
    An IAM policy is a JSON-based document that explicitly defines what actions are allowed or denied on which resources and under what conditions Policies are attached to users, groups, or roles to define their permissions.

    > **Analogy**: In firewall configuration, you define rules like “allow port 22 from IP range X” or “deny all traffic except port 80.” IAM policies work similarly, but for AWS resources: they define rules like “allow starting EC2 instances in us-west-2” or “deny deleting S3 buckets.”

    Policies are the fine-grained mechanism by which all AWS access is controlled, and mastering them is critical to secure cloud operation.


### Create an IAM Group

Let's have a look at how to create an IAM group with granular permission. To this end, connect to your fresh AWS with you master password, or with your personal credentials, if you have already a user for the account. 

Once logged in you should search the 

1. Search for **IAM** in the AWS Console search bar and click.  
    <img src="assets/pics/000_iam_user_roles_and_policies/IAM.png" alt="Alt text" width="500rem" style="padding-top: .5rem;"/>

2. In the left menu, click User groups.  
    <img src="assets/pics/000_iam_user_roles_and_policies/user_group_category.png" alt="Alt text" width="200rem" style="padding-top: .5rem;"/>

3. Click Create group.  
    <img src="assets/pics/000_iam_user_roles_and_policies/create_group.png" alt="Alt text" width="200rem" style="padding-top: .5rem;"/>

4. Name the group (e.g., TutorialNetworkingAdmins).

5. Under **Attach permissions policies**, search and select:
   - `AmazonEC2FullAccess`
   - `AmazonVPCFullAccess`
6. Click **Create group**.

This group now has full permissions to EC2 and VPC operations, suitable for most networking operations. You can later replace these managed policies with a custom scoped one.

To create the user group with all access needed for this tutorial, you will need to add the permissions listed out in the [required permissions file](./requried_permissions.md)




### Create an IAM User and Assign It to the Group

Once the group is created, we can start assigning users to it. In the IAM section, follow these instructions:

**Creating the user:**  
1. In the IAM console sidebar, click **Users**.  
    <img src="assets/pics/000_iam_user_roles_and_policies/users_category.png" alt="Users tab" width="220rem" style="padding-top: .5rem;"/>

2. Click **Create User**.  
    <img src="assets/pics/000_iam_user_roles_and_policies/create_user.png" alt="Add user button" width="220rem" style="padding-top: .5rem;"/>

3. Enter a username (e.g., `tutorial-user`).  
    Here, you can also deside wether the user should be able to access the Web UI AWS Console, or wether it operates as a server only.

5. Click **Next**.
6. Choose **Add user to group**, then select the group you created earlier.
    <img src="assets/pics/000_iam_user_roles_and_policies/set-permissions.png" alt="Enter username" width="700rem" style="padding-top: .5rem;"/>  
7. Continue with default tags and click **Create user**.

 
You have now created the user and granted it the permissions from the group we created earlier. Next, return to the Users tab of the IAM section and click the users name. 

**Obtaining your access key:**  
1. Then select the **Security Credentials** pane.  
    <img src="assets/pics/000_iam_user_roles_and_policies/security_credentials.png" alt="Enter username" width="500rem" style="padding-top: .5rem;"/>  
Here you can add MFA and generate the account key, which we will use for API access.
2. Click on **Create access key** and select the local code option. Then click **Next**
    <img src="assets/pics/000_iam_user_roles_and_policies/local_code_option.png" alt="Enter username" width="500rem" style="padding-top: .5rem;"/>  
3. Enter the description tag and select *Create Access Key*. You are now provided with both:
   - Access key (allows AWS to identify who is connecting like a username)
   - Secret Access Key (the password)
   Make sure to store both informations securely. We will use them to authenticate through the SDKs in a later section.



### Creating a Role in the AWS Console

Other than users, roles grant access without the use of a password, secrets or permanent associations. They are usefull when granting automated or cross account permissions. An example would be giving an EC2 instance to S3 buckets.
We’ll walk through creating a role that grants permission to access S3 and can be assumed by an EC2 instance.

1. In the sidebar, click **Roles**.  
    <img src="assets/pics/000_iam_user_roles_and_policies/role_category.png" alt="Roles sidebar" width="220rem" style="padding-top: .5rem;"/>

2. Click **Create role**.  
    <img src="assets/pics/000_iam_user_roles_and_policies/create_role.png" alt="Create role button" width="200rem" style="padding-top: .5rem;"/>

3. Under **Trusted entity type**, select **AWS service**, and under **Use Case** choose **EC2**. Then press next.  

4. Search for the policy `AmazonS3ReadOnlyAccess`. Select it and press next.

5. Click **Next**, then name the role (e.g., `EC2S3ReadOnlyRole`) and optionally add a description.  

6. Click **Create role**.

You will be able to assign this Role to EC2 instances, to access S3 storage buckets. Both are concepts we will explore in later notebooks.

!TODO section about roles

## Understanding the AWS CLI and Python SDKs

Before we authenticate and interact with AWS services, it’s important to understand the tools we’ll be using: the **AWS CLI** and **boto3**, AWS’s official Python SDK.

These tools allow you to create, configure, and inspect AWS resources from your terminal or Python code: automating infrastructure, managing cloud operations, and integrating with applications.

### AWS CLI

The AWS Command Line Interface (CLI) is a powerful tool for interacting with AWS using shell commands. It's ideal for scripting, infrastructure provisioning, and quick configuration tasks.
You can use it to script your infrastructure and perform all action you usually would within the console. 

You can simply install it using conda:

In [ ]:
%conda install -c conda-forge awscli

Test your install by running:

In [2]:
!aws --version

aws-cli/1.38.18 Python/3.11.11 Windows/10 botocore/1.37.18


### boto3: AWS SDK for Python

**boto3** is the official AWS SDK for Python. It lets you script and automate AWS resources from Python, giving you full programmatic access to the same APIs used by the AWS CLI.

Use it can use it to write infrastructure scripts in notebooks, automate resource creation and cleanup, or interact with S3, EC2, Lambda, and more.

You can simply install it through pip:

In [ ]:
%pip install boto3 python-dotenv

Test your install by running:

In [3]:
import boto3
print(boto3.__version__)

1.37.18


## Authenticating to AWS: Access Keys and CLI Configuration

To interact with AWS services using SDKs like boto3 or the AWS CLI, your session must be authenticated. AWS offers several methods for authentication, with varying levels of security, flexibility, and automation.

This section explains how to authenticate using **access keys**, how to securely store them, and how to verify that authentication works with boto3.

###  Default Profile with AWS CLI
If you run `aws configure` **without** `--profile`, it creates a `default` profile. Once done, will not need to specify anything in `boto3`. The CLI dialogue will query you for your access key ID and secret access key, both of witch you obtained when creating a user earlier in the tutorial.

In [4]:
!aws configure

^C


If the prompt does not work in the notebook, try in the CLI.

You can try wether the authentication worked, by running the following code. If you obtain a HTTPStatusCode of 200, you have benn successfully authenticated.

In [7]:
import boto3
import json

# Uses default profile and region
ec2 = boto3.client("ec2")
print(json.dumps(ec2.describe_instances(), indent=4))

{
    "Reservations": [],
    "ResponseMetadata": {
        "RequestId": "1c801485-a1c1-4a42-a9d1-11604db00570",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "x-amzn-requestid": "1c801485-a1c1-4a42-a9d1-11604db00570",
            "cache-control": "no-cache, no-store",
            "strict-transport-security": "max-age=31536000; includeSubDomains",
            "content-type": "text/xml;charset=UTF-8",
            "content-length": "219",
            "date": "Sun, 23 Mar 2025 15:09:02 GMT",
            "server": "AmazonEC2"
        },
        "RetryAttempts": 0
    }
}


### Configure a Named Profile Using the AWS CLI

Other than specifying the default profile, you can create a named profile. This is the *recommended approach for local development*. AWS stores your credentials securely in `~/.aws/credentials` and makes them accessible to both the AWS CLI and `boto3`.

Run the following in your terminal:

In [8]:
!aws configure --profile tutorial-user

^C


This creates two files:
- `~/.aws/credentials` – stores your keys
- `~/.aws/config` – stores region and output preferences

In [11]:
import boto3

# Use the named profile created with AWS CLI
session = boto3.Session(profile_name="tutorial-user")
ec2 = session.client("ec2")
ec2.describe_instances()

{'Reservations': [],
 'ResponseMetadata': {'RequestId': 'c35229fc-f1c5-4ed5-b55a-a7c78911cbcd',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c35229fc-f1c5-4ed5-b55a-a7c78911cbcd',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '219',
   'date': 'Sun, 23 Mar 2025 15:13:04 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

### Use Environment Variables

This method is useful for CI/CD pipelines, temporary testing, or notebook environments.

You can export the credentials into your shell, or loally define them in a .env file, which you should explicitely exclude from any form of version control

```env
AWS_ACCESS_KEY_ID=your-access-key-id
AWS_SECRET_ACCESS_KEY=your-secret-access-key
AWS_DEFAULT_REGION=us-east-1
```

And load them in Python using `python-dotenv`:

In [13]:
import os
from dotenv import load_dotenv
import boto3

load_dotenv()

# Automatically reads credentials from environment
session = boto3.Session()
ec2 = session.client("ec2")
ec2.describe_instances()

{'Reservations': [],
 'ResponseMetadata': {'RequestId': '2d8d23da-8071-40a3-86de-f72726ca0eb0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2d8d23da-8071-40a3-86de-f72726ca0eb0',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '219',
   'date': 'Sun, 23 Mar 2025 15:15:11 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

### Security Tips

- Prefer named profiles for local work, roles for services, and env vars for CI/CD.
- Never hardcode credentials in source files.
- Use MFA and rotate access keys regularly.
- Audit key usage with IAM Access Analyzer and CloudTrail.
